In [1]:
init()

Antidote session created. Connected to Antidote node 1.

In [3]:
switchAntidote(1)

Connected to Antidote 1

In [0]:
connectAntidotes()

In [1]:
disconnectAntidotes()

1

In [4]:
registerKey = getLWRegisterKey("r1")

LWWREG_r1

In [5]:
regUp = assignLWRegister(registerKey, "rval5")
applyUpdate(regUp)

Update to key '<ByteString@77154dd9 size=2>' applied.

In [23]:
read(registerKey)

rval3

In [7]:
tx = startTransaction()

eu.antidotedb.client.InteractiveTransaction@3a5025ba

In [8]:
regUp2 = assignLWRegister(registerKey, "rval2")
addToTransaction(tx, regUp2)
commitTransaction(tx)

Cell returned null.